In [4]:
numstr = ""
for i in range(1001):
    numstr += str(i)

for i in range(10):
    print(f"{i}: {numstr.count(str(i))}")


0: 193
1: 301
2: 300
3: 300
4: 300
5: 300
6: 300
7: 300
8: 300
9: 300


In [1]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# 加载模型与分词器
model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
model.eval()

# 输入句子
sentence = "窗[MASK]西岭千秋雪，门泊东吴万里船"
masked_sentence = sentence.replace("[MASK]", tokenizer.mask_token)

# 编码
inputs = tokenizer(masked_sentence, return_tensors="pt")
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

# 模型预测
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# 获取[MASK]位置预测结果
mask_logits = logits[0, mask_token_index, :]
top_k = 10
top_k_indices = torch.topk(mask_logits, top_k, dim=1).indices[0].tolist()

# 转为token与概率
softmax = torch.nn.functional.softmax(mask_logits, dim=1)
top_k_probs = softmax[0, top_k_indices].tolist()
top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices)

# 输出结果
for rank, (token, prob) in enumerate(zip(top_k_tokens, top_k_probs), 1):
    print(f"{rank}. {token} - {prob:.4f}")


2025-07-26 23:57:53.320701: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-26 23:57:53.343704: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-26 23:57:53.501335: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-26 23:57:53.637639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753545473.754447    5563 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753545473.78

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1. 开 - 0.1458
2. 临 - 0.0499
3. 观 - 0.0314
4. 怀 - 0.0302
5. 前 - 0.0210
6. 封 - 0.0196
7. 窗 - 0.0179
8. 看 - 0.0175
9. 倚 - 0.0174
10. 销 - 0.0154


In [2]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# 加载模型和分词器
model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
model.eval()

# 交互式输入
while True:
    sentence = input("请输入包含一个 X 的中文句子（输入 'exit' 退出）：\n> ")
    if sentence.lower() == 'exit':
        break
    if "X" not in sentence:
        print("❌ 句子中必须包含一个 X。请重试。\n")
        continue

    # 替换为模型的 mask token
    masked_sentence = sentence.replace("X", tokenizer.mask_token)

    # 编码
    inputs = tokenizer(masked_sentence, return_tensors="pt")
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

    # 模型预测
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 获取 X 位置的 logits 并 softmax
    mask_logits = logits[0, mask_token_index, :]
    softmax = torch.nn.functional.softmax(mask_logits, dim=1)

    # Top 10 预测
    top_k = 10
    top_k_indices = torch.topk(mask_logits, top_k, dim=1).indices[0].tolist()
    top_k_probs = softmax[0, top_k_indices].tolist()
    top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices)

    # 输出
    print(f"\n🎯 输入句子：{sentence}")
    for i, (token, prob) in enumerate(zip(top_k_tokens, top_k_probs), 1):
        print(f"{i}. {token}  -  概率: {prob:.4f}")
    print()


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



🎯 输入句子：氓之蚩蚩，抱布贸丝。匪来贸丝，来即我谋。送子涉淇，至于顿丘。匪我愆期，子无良媒。将子无怒，秋以为期。  乘彼垝垣，以望复关。不见复关，泣涕涟涟。既见复关，载笑载言。尔卜尔筮，体无咎言。以尔车来，以我贿迁。桑之未落，其叶X若。于嗟鸠兮！无食桑葚。于嗟女兮！无与士耽。士之耽兮，犹可说也。女之耽兮，不可说也。  桑之落矣，其黄而陨。自我徂尔，三岁食贫。淇水汤汤，渐车帷裳。女也不爽，士贰其行。士也罔极，二三其德。  三岁为妇，靡室劳矣。夙兴夜寐，靡有朝矣。言既遂矣，至于暴矣。兄弟不知，咥其笑矣。静言思之，躬自悼矣。  及尔偕老，老使我怨。淇则有岸，隰则有泮。总角之宴，言笑晏晏，信誓旦旦，不思其反。反是不思，亦已焉哉！
1. 不  -  概率: 0.0865
2. 若  -  概率: 0.0570
3. 如  -  概率: 0.0562
4. 未  -  概率: 0.0483
5. 犹  -  概率: 0.0307
6. 相  -  概率: 0.0305
7. 自  -  概率: 0.0274
8. 亦  -  概率: 0.0231
9. 无  -  概率: 0.0203
10. [UNK]  -  概率: 0.0180

